# How to generate names with RNNs

This tutorial is available at https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html and it can be seen as a follow up to https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html.

We use a character-level recurrent nn to generate names given a language.

## The data
The data is available at https://download.pytorch.org/tutorial/data.zip. The folder `data/names` contains 18 Unicode text files, one for each of 18 languages, with names such as `[Language].txt`.

With the following code we read the files, convert the data to ASCII and create a dictionary with shape `{language: [name1, name2, ...]}`:

In [1]:
# packages
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

In [2]:
# turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
all_letters = string.ascii_letters + " .,;'-" # our alphabet
n_letters = len(all_letters) + 1 # plus 1 because of EOS marker

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# for example:
print(unicodeToAscii("O'Néàl"))

O'Neal


In [3]:
# read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# get a list of paths for each of the files found with a certain pattern (see below)
def findFiles(path): return glob.glob(path)

# build the category_lines dictionary, a list of lines (i.e. names) per category (i.e. languages)
category_lines = {}
all_categories = []
for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

print('# categories:', n_categories, all_categories)

# categories: 18 ['Greek', 'German', 'Portuguese', 'Irish', 'Scottish', 'Czech', 'English', 'Vietnamese', 'Polish', 'Korean', 'French', 'Spanish', 'Arabic', 'Chinese', 'Dutch', 'Japanese', 'Italian', 'Russian']


## The model

We build a character-level recurrent nn. The input is a pair `<language, name>` where `name` is encoded as a matrix where each row is the tensor corresponding to a letter in the name. The output is a probability distribution over letters (roughly, the probability of the next letter given the previous).

(A picture of the model is available at https://i.imgur.com/jzVrf7f.png.)

The source of recurrence is the fact that a hidden layer computed from the input at a gvien stage (i.e. a letter in the name) is fed to the nn together with the input pair itself at the following stage (i.e. the following letter); moreover, the output at each stage is also fed to the nn as input in the following stage.

In [4]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size) # linear transf. input-to-hidden
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size) # input-to-(intermediate-)output
        self.o2o = nn.Linear(hidden_size + output_size, output_size) #(intermediate-)output-to-(actual-)output
        self.dropout = nn.Dropout(0.1) # the intermediate output is filtered with dropout
        self.softmax = nn.LogSoftmax(dim=1) # then transformed into prob. distribution, the actual output

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1) # input pair and hidden layer are combined here
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)